In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import decomposition

In [2]:
# Storing the file path in 'loc' variable
loc = (r"C:\Users\Minet\Documents\Repositories\Dev10_Final_Project\Datasets\MLModelDatasetPowerBI.csv")
# Importing the file from the file path contained in 'loc' into 'df' 
df = pd.read_csv(loc)

In [3]:
df = df[['GraduationPercentageRate','HouseholdIncome','FundingPerStudent', 'DropoutPercentageRate']]
df.head()

,GraduationPercentageRate,HouseholdIncome,FundingPerStudent,DropoutPercentageRate
0,90,59328.0,23833.02,3
1,92,63986.4,14013.61,5
2,83,54818.3,24128.02,10
3,95,76903.8,17552.56,3
4,71,61049.2,22382.65,12


In [4]:
# nysd = datasets.load_nystatedistricts()
features=['HouseholdIncome','FundingPerStudent', 'DropoutPercentageRate']
# features=['HouseholdIncome']
target=['GraduationPercentageRate']

X = df[features]
print(X.shape)

y = df.iloc[:, 0]
print(y.shape)

(610, 3)
(610,)


In [15]:
# Create training and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
std_feats = scaler.fit(X_train)

# X_scaled = scaler.fit_transform(X)
# X_scaled = pd.DataFrame(df = X_scaled, columns = X.columns)

# reduce the dimension of features by creating new features which have most of the varience of the original data
pca = decomposition.PCA()

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Create an instance of Lasso Regression implementation
lasso = Lasso(alpha=0.0)

(457, 3)
(153, 3)
(457,)
(153,)


In [17]:
# helps us by passing modules one by one through GridSearchCV for which we want to get the best parameters.
pipe = Pipeline(steps=[("scaler", scaler),
                           ("pca", pca),
                           ("lasso", lasso)])

# for GridSearchCV

# for pca, n_components signifies the number of components to keep after reducing the dimension.
n_components = list(range(1,X.shape[1]+1,1))

# Logistic Regression requires two parameters "normalize" and "selection" to be optimised by GridSearchCV.
normalize = [True, False]
selection = ["cyclic", "random"]
alpha = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
fit_intercept = [True, False]

# creating a dictionary to set all the parameters options for different modules.
parameters = dict(pca__n_components=n_components,
                      lasso__normalize=normalize,
                      lasso__selection=selection,
                      lasso__alpha=alpha,
                      lasso__fit_intercept=fit_intercept)

# Making an object clf_GS for GridSearchCV and fitting the X and y
clf = GridSearchCV(pipe, parameters)
clf.fit(X, y)

# print the results. It will give the values of hyperparameters as a result.
print("Best Number Of Components:", clf.best_estimator_.get_params()["pca__n_components"])
print(clf.best_estimator_.get_params()["lasso"])

CV_Result = cross_val_score(clf, X, y, cv=10, n_jobs=-1, scoring="r2")
print(CV_Result)
print(CV_Result.mean())
print(CV_Result.std())

E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\pipeline.py:382: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.p

E:\Anaconda\lib\site-packages\sklearn\pipeline.py:382: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.757e+03, tolerance: 3.278e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\li

E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\pipeline.py:382: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You mi

E:\Anaconda\lib\site-packages\sklearn\pipeline.py:382: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.952e+06, tolerance: 3.920e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\li

E:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.942e+06, tolerance: 3.907e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\pipeline.py:382: UserWarning: Wit

...

E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to 

E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline impor

E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to 

E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
E:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline impor

Best Number Of Components: 3
Lasso(alpha=0.0, normalize=True)
[0.6459853  0.09316907 0.8208703  0.68848555 0.81277301 0.6486641
 0.70782531 0.69431586 0.48825047 0.70019869]
0.6300537667892981
0.1993146745687222


In [7]:
# Fit the Lasso model
lasso.fit(X_train, y_train)

Lasso()

In [8]:
y_pred = lasso.predict(X_test)

In [9]:
y_pred

array([93.18407578, 86.85193913, 95.81290969, 93.90037901, 79.34651052,
       85.55426739, 90.36790041, 91.13288814, 95.58254203, 88.63206205,
       85.73799919, 85.7408235 , 87.87393007, 79.32487193, 32.40663706,
       76.46723876, 93.35023521, 91.78692729, 89.72082495, 81.63738954,
       92.06869259, 91.87470202, 84.69714234, 85.89545705, 88.21131383,
       86.95785674, 85.80631324, 88.18873975, 83.6188612 , 91.51124989,
       90.38542122, 83.41174904, 71.79809366, 86.89738031, 96.84661624,
       88.10064159, 90.66405846, 90.4000299 , 93.70550374, 90.41633082,
       88.34771492, 89.22182943, 89.20679196, 94.32019761, 88.05171651,
       92.19263896, 82.31192136, 86.67205749, 88.43006563, 95.40052078,
       96.45119456, 96.0429247 , 92.72566249, 94.73284242, 85.04553933,
       88.10296731, 74.69095482, 92.08303129, 95.4458804 , 92.6865682 ,
       76.64815117, 91.9362803 , 91.39072378, 95.03649126, 91.88721832,
       83.67793158, 77.20753884, 84.88317167, 90.6579165 , 94.32

In [10]:
# Create the model score
lasso.score(X_test, y_test)

0.7601530507815153

In [11]:
# dump(lasso, 'C:/Users/Minet/Documents/Repositories/Dev10_Final_Project/Datasets/joblib.model')